# 1. Symplectic representation

## 1.1. On Humphries generators

### Sp_repr class and functions

In [1]:
from sage.all import *

In [2]:
%display latex

In [3]:
class Sp_repr:
    #--- 2x2 ---
    I = identity_matrix(2)
    O = zero_matrix(ZZ,2)
    J = matrix([[0,1],[-1,0]])
    #---
    L = matrix([[1,0],[1,1]])
    K = matrix([[0,-1],[0,0]])
    #--- 4x4 ---
    J4 = block_matrix([[J, O], [O, J]])
    # J4 = J4_bm.subs([(J, evJ)]).as_explicit()
    #---
    CHR_2_MTX = {'a': block_matrix([[L.inverse(), O], [O, I]]),
                 'b': block_matrix([[L.T, K], [K, L.T]]),
                 'c': block_matrix([[I, O], [O, L.inverse()]]),
                 'd': block_matrix([[I, O], [O, L.T]]),
                 'f': block_matrix([[L.T, O], [O,I]])}
    #---
    
    def __init__(self, l:str):
        self.loop = l
        self.matrix = self.CHR_2_MTX.get(l) if l.islower() else self.CHR_2_MTX.get(l.lower()).inverse()

In [4]:
def is_Sp(M):
    J = Sp_repr.J4
    return M.transpose()*J*M == J

list(map(lambda l: is_Sp(Sp_repr(l).matrix), ['a','b','c','d','f', 'A'])) 

[True, True, True, True, True, True]

## 1.2. The symplectic images of Abcd and abcD

In [5]:
a,b,c,d,f = tuple(map(lambda l: Sp_repr(l).matrix, ['a','b','c','d','f']))
J4 = Sp_repr.J4

a,b,c,d,f, J4 

(
[ 1  0| 0  0]  [ 1  1| 0 -1]  [ 1  0| 0  0]  [1 0|0 0]  [1 1|0 0]
[-1  1| 0  0]  [ 0  1| 0  0]  [ 0  1| 0  0]  [0 1|0 0]  [0 1|0 0]
[-----+-----]  [-----+-----]  [-----+-----]  [---+---]  [---+---]
[ 0  0| 1  0]  [ 0 -1| 1  1]  [ 0  0| 1  0]  [0 0|1 1]  [0 0|1 0]
[ 0  0| 0  1], [ 0  0| 0  1], [ 0  0|-1  1], [0 0|0 1], [0 0|0 1],

[ 0  1| 0  0]
[-1  0| 0  0]
[-----+-----]
[ 0  0| 0  1]
[ 0  0|-1  0]
)

In [6]:
A, B, C, D, F = (k.inverse() for k in [a,b,c,d,f])
A,B,C,D,F

(
[1 0 0 0]  [ 1 -1  0  1]  [1 0 0 0]  [ 1  0  0  0]  [ 1 -1  0  0]
[1 1 0 0]  [ 0  1  0  0]  [0 1 0 0]  [ 0  1  0  0]  [ 0  1  0  0]
[0 0 1 0]  [ 0  1  1 -1]  [0 0 1 0]  [ 0  0  1 -1]  [ 0  0  1  0]
[0 0 0 1], [ 0  0  0  1], [0 0 1 1], [ 0  0  0  1], [ 0  0  0  1]
)

In [7]:
M = a*b*C*D*f
display(M)
N = a*b*b*C*D*f
display(N)

list(map(is_Sp, [M, N]))

[ 1  2 -1  0]
[-1 -1  1  0]
[ 0 -1  2 -1]
[ 0  0  1  0]

[ 1  3 -2  0]
[-1 -2  2  0]
[ 0 -2  3 -1]
[ 0  0  1  0]

[True, True]

# 2. The ideal for fibered knots

### The charactoristic polynomial of M (and N)

In [8]:
R.<alpha> = PolynomialRing(ZZ)
cp_M = M.charpoly('t'); display(cp_M, cp_M.factor())
cp_N = N.charpoly('t'); #display(cp_N)

print(f"cp_M == cp_N ?   --> {cp_M == cp_N}")

t^4 - 2*t^3 + 3*t^2 - 2*t + 1

(t^2 - t + 1)^2

cp_M == cp_N ?   --> True


In [9]:
K.<alpha> = NumberField(cp_M)
OK = K.ring_of_integers()
print("整数基:", OK.basis())
print("類数:", OK.class_number())

U = UnitGroup(K)
print(U.gens_values())

G = K.galois_group()
tau = K.hom([1/alpha],K); print(f"{tau(alpha) = }")

ValueError: defining polynomial (t^4 - 2*t^3 + 3*t^2 - 2*t + 1) must be irreducible

In [46]:
fac = cp_M.factor()[0][0]
K = NumberField(fac, names=('alpha'))
root = K.gen()
U = UnitGroup(K); print(U.gens_values())
tau = K.hom([1/alpha],K); print(f"{tau(alpha) = }")

[alpha]
tau(alpha) = -alpha + 1


In [34]:
def ev_wrt_a(X):
    X_K = X.change_ring(K)
    evlist = X_K.eigenvectors_right();
    v = [v[1][0] for v in evlist if v[0] == root][0]
    if X_K*v == root*v:
        return v
    else:
        return None

### Eigenvectors of M and N with respect to the eigen value $a$

In [36]:
v, w = [ev_wrt_a(X) for X in (M,N)]
display(v, w)
print("------- Check -------")
print(M*v == root*v, N*w == root*w)

(1, alpha - 1, alpha - 1, alpha)

(1, alpha - 1, alpha - 1, alpha)

------- Check -------
True True


### The values of M and N

In [51]:
df = diff(cp_M) #fac) #
Del = df(alpha)/alpha; display(Del)

(4*alpha^3 - 6*alpha^2 + 6*alpha - 2)/alpha

In [52]:
def Va(v):
    tau_v = vector([tau(e) for e in v])
    aD = v.dot_product(J4*tau_v)
    return aD/Del

In [53]:
[Va(vec) for vec in [v,w]]

[0, 0]

In [54]:
(v, Va(v)), (w, Va(w))

(((1, alpha - 1, alpha - 1, alpha), 0), ((1, alpha - 1, alpha - 1, alpha), 0))

In [55]:
c = Va(v)/Va(w)
display(c, c == tau(c))

ZeroDivisionError: fraction field element division by zero

In [ ]:
# --- 判定処理 ---
def check_decomposition(x, sigma, O):
    """
    x が u*sigma(u) の形に分解できるか検証する
    """
    # x が代数的整数であるか確認
    if x not in O:
        print(f"警告: {x} は代数的整数ではありません。")
        
    print(f"\n--- 元 x = {x} の検証 ---")
    
    # 必要条件: x が共役写像 σ で不変であるかチェック
    if sigma(x) != x:
        print(f"不変性チェック失敗: σ(x) = {sigma(x)} は x と異なります。")
        print("したがって、x は u * σ(u) の形には分解できません。")
        return False
    
    print(f"不変性チェック成功: σ(x) = {sigma(x)} で、x と一致します。")
    print("分解できる可能性があります。")
    
    # # u * σ(u) = x となる u の存在を調べる
    # # この部分の実装は、一般には困難
    # # 幸いにも、このケースでは K^* のコホモロジー群が自明なため、
    # # 不変元は常に u*σ(u) の形に書けます。
    # # よって、不変性チェックが成功すれば、分解可能と判定します。
    # # 実際には、x が代数的整数であるかどうかのチェックも重要です。
    # print("理論上、不変元であるため、単数 u が存在して分解できます。")
    
    # ノルムを計算
    x_norm = x.norm()
    print(f"ノルム: {x_norm}")

    # 平方数であるかチェック
    if is_square(x_norm):
        print("ノルムは平方数です。分解できる可能性があります。")
    else:
        print("ノルムは平方数ではありません。分解できません。")
        
    return True

# 検証を実行
check_decomposition(c, tau, OK)

In [ ]:
# 第10円分体 K を定義
K.<a> = CyclotomicField(10)

# a を 1/a に送る共役写像 σ を定義
sigma = K.hom([1/a], K)

# 調べる元 x = a^3 - a^2
x = a^3 - a^2

# x の共役 σ(x) を計算
sigma_x = sigma(x)

print(f"元 x = {x}")
print(f"共役 σ(x) = {sigma_x}")

# x と σ(x) が一致するか比較
if x == sigma_x:
    print("\nx は σ-不変です。分解できる可能性があります。")
else:
    print(f"\nx と σ(x) が一致しません。")
    print("したがって、x は u * σ(u) の形には分解できません。")

In [ ]:
from itertools import product
R = range(100)
frag = False

u0, u1 = U.gens_values()
for i,j in product(R,R):
    for e, d in product([1,-1],[1,-1]):
        u = (u0**(i*e))*(u1**(j*d))
        frag = u*tau(u) == c
        if frag:
            print((i*e,j*d), u)
            break
    if frag: break                    

### Scratch

In [ ]:
#print(v, type(v))
display(a+1, (a+1).subs({a: 1/a}))
display(v, v.subs({a: 1/a}), J4*(v.subs({a: 1/a})))
# v.dot_product(J4*v)

In [ ]:
tv = vector([1, 1/a^3+1/a-1, -1/a^3, 1/a^2])
display(tv)
alpha = v.dot_product(J4*tv)
display(alpha)

In [ ]:
(a-2)*(1/a-2)

In [ ]:
vv = vector([1, a, a^2, a^3])
tvv = vector([1, 1/a, 1/(a^2), 1/(a^3)])
display(vv, tvv)

aDel = bilinear_form(vv, tvv, J4)
display(aDel)

cvv = (1/aDel)*(J4*tvv)
display(cvv)

T = 10*(identity_matrix(4)+matrix([[0,0,0,-1],[0,0,0,1],[0,0,0,1],[0,0,0,0]]))
cvv = T*cvv; display(cvv)
T2 = (identity_matrix(4)+matrix([[0,0,0,0],[0,0,0,0],[0,-1,0,0],[2,2,0,0]]))
cvv = T2*cvv; display(cvv)
T3 = (identity_matrix(4)+matrix([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,1,0]]))
cvv = T3*cvv; display(cvv)

In [ ]:
vv = vector([1, a, a^2, a^3])
tvv = vector([1, 1/a, 1/(a^2), 1/(a^3)])
display(vv, tvv)

aDel = bilinear_form(vv, tvv, J4)

display(aDel, aDel == 2*(1/a-a))    

In [ ]:
t = var('t')
df = diff(cp_M, t)

Del = df(a)/a

display(Del)

In [ ]:
VA = 2*(1/a-a)/Del #aDel/Del
display(VA, VA == VA)